In [ ]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import numpy as np

# Function to classify a new email
def classify_user_email(email_text, vectorizer, model):
    """Classifies a given email text as 'spam' or 'not spam'."""
    cleaned_email = email_text.lower()
    cleaned_email = re.sub('<.*?>', '', cleaned_email)
    cleaned_email = re.sub('[^\w\s]', '', cleaned_email, flags=re.UNICODE)
    cleaned_email = cleaned_email.encode('ascii', 'ignore').decode('ascii')
    email_vector = vectorizer.transform([cleaned_email])
    prediction = model.predict(email_vector)
    return "Spam" if prediction[0] == 'spam' else "Not Spam"

# --- Main Classification Process ---
try:
    # Data Loading and Initial Cleaning
    df = pd.read_csv('Book1.csv', encoding='latin-1')
    df.rename(columns={df.columns[1]: 'email_text'}, inplace=True)
    df.dropna(inplace=True)
    df.drop_duplicates(inplace=True)

    # Text Cleaning
    df['email_text'] = df['email_text'].str.lower()
    df['email_text'] = df['email_text'].apply(lambda x: re.sub('<.*?>', '', str(x)))
    df['email_text'] = df['email_text'].str.replace('[^\w\s]', '', regex=True)
    df['email_text'] = df['email_text'].str.encode('ascii', 'ignore').str.decode('ascii')

    # Data Preparation (TF-IDF Vectorization)
    vectorizer = TfidfVectorizer(max_features=5000, stop_words='english', ngram_range=(1, 2))
    X = vectorizer.fit_transform(df['email_text'])
    y = df['ham'] # Assuming the target column is 'ham'

    # Train/Test Split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Model Training (using a reasonable alpha value)
    # Based on previous analysis, 0.5 was a good alpha, using that directly
    nb_classifier = MultinomialNB(alpha=0.5)
    nb_classifier.fit(X_train, y_train)

    # Model Evaluation
    y_pred = nb_classifier.predict(X_test)

    print("\n--- Model Evaluation ---")
    # Display Accuracy
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy: {accuracy:.4f}")

    # Display Confusion Matrix (shows errors: False Positives and False Negatives)
    cm = confusion_matrix(y_test, y_pred, labels=['ham', 'spam'])
    print("\nConfusion Matrix:")
    print(cm)
    print("Interpretation: ")
    print(f"  - True Negatives (Ham correctly classified as Ham): {cm[0, 0]}")
    print(f"  - False Positives (Ham incorrectly classified as Spam): {cm[0, 1]}")
    print(f"  - False Negatives (Spam incorrectly classified as Ham): {cm[1, 0]}")
    print(f"  - True Positives (Spam correctly classified as Spam): {cm[1, 1]}")

    # Optional: Display Classification Report for more detailed metrics
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred, target_names=['ham', 'spam']))


    # --- User Input for Classification ---
    print("\n--- Email Classification ---")
    user_email = input("Enter an email to classify (or type 'quit' to exit): ")
    while user_email.lower() != 'quit':
        classification = classify_user_email(user_email, vectorizer, nb_classifier)
        print(f"The email is classified as: {classification}")
        user_email = input("\nEnter another email to classify (or type 'quit' to exit): ")


except FileNotFoundError:
    print("Error: 'Book1.csv' not found. Please ensure the file exists in the current directory.")
except pd.errors.EmptyDataError:
    print("Error: 'Book1.csv' is empty.")
except pd.errors.ParserError:
    print("Error: Unable to parse 'Book1.csv'. Please check the file format.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")


--- Model Evaluation ---
Accuracy: 0.9826

Confusion Matrix:
[[898   1]
 [ 17 118]]
Interpretation: 
  - True Negatives (Ham correctly classified as Ham): 898
  - False Positives (Ham incorrectly classified as Spam): 1
  - False Negatives (Spam incorrectly classified as Ham): 17
  - True Positives (Spam correctly classified as Spam): 118

Classification Report:
              precision    recall  f1-score   support

         ham       0.98      1.00      0.99       899
        spam       0.99      0.87      0.93       135

    accuracy                           0.98      1034
   macro avg       0.99      0.94      0.96      1034
weighted avg       0.98      0.98      0.98      1034


--- Email Classification ---
Enter an email to classify (or type 'quit' to exit): i love you
The email is classified as: Not Spam
